<a href="https://colab.research.google.com/github/vishh17/NLP/blob/master/IMDB_movie_reveiw_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install ktrain

In [4]:
# clone data from git

!git clone https://github.com/vishh17/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text


In [9]:
df_train= pd.read_excel('/content/NLP/train.xlsx', dtype= str)

In [10]:
df.head(5)

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [12]:
df_test= pd.read_excel('/content/NLP/test.xlsx', dtype= str)
df_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [13]:
df_train.shape, df_test.shape

((25000, 2), (25000, 2))

In [15]:
df_train.isnull().sum()

Reviews      0
Sentiment    0
dtype: int64

In [16]:
df_test.isnull().sum()

Reviews      0
Sentiment    0
dtype: int64

In [26]:
df_train['Sentiment'].value_counts() 

pos    12500
neg    12500
Name: Sentiment, dtype: int64

In [27]:
df_test['Sentiment'].value_counts() 

pos    12500
neg    12500
Name: Sentiment, dtype: int64

In [30]:
(X_train, y_train),(X_test, y_test), preprocess = text.texts_from_df(train_df= df_train, 
                                                                text_column='Reviews',
                                                                label_columns= 'Sentiment',
                                                                val_df= df_test,
                                                                maxlen= 400,
                                                                ngram_range=1,
                                                                preprocess_mode='bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [42]:
X_train[0].shape

(25000, 400)

In [45]:
X_test[0].shape

(25000, 400)

In [38]:
model= text.text_classifier(name= 'bert', 
                            train_data= (X_train, y_train),
                            preproc= preprocess, 
                            verbose=3)

Is Multi-Label? False
maxlen is 400
done.


In [48]:
X_test


[array([[  101,  2040,  2052, ...,  2411,  1998,   102],
        [  101,  2044,  9301, ...,     0,     0,     0],
        [  101,  1045,  3473, ...,     0,     0,     0],
        ...,
        [  101,  4151, 10231, ...,     0,     0,     0],
        [  101,  5119,  2075, ...,     0,     0,     0],
        [  101, 12524,  1998, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [50]:
# Find  the learner
# Complile  the model
learner= ktrain.get_learner(model= model, 
                            train_data=(X_train, y_train),
                            val_data= (X_test, y_test), 
                            batch_size= 6)

In [ ]:
# To get the optimatal learning rate as per the problem
# this require heavy compution, so it need larget the find a best learner

# https://nbviewer.jupyter.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-04-text-classification.ipynb

# learner.lr_find()
# learner.plot()

In [ ]:
# we are using the assumed learner for  our  problem is 2e-5


In [51]:
# fit the  model
learner.fit_onecycle(lr= 2e-5,
                     epochs= 1,
                     verbose= 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 6010s 240ms/sample - loss: 0.2510 - accuracy: 0.8974


KeyboardInterrupt: ignored

In [52]:
learner

In [54]:
predictor= ktrain.get_predictor(learner.model,preproc=preprocess)

Testing the data


In [57]:
data= ['this movie was good, but I didnot like the acting',
       'I dint not understand the story of the movie.',
       "want  to see  the squeal  of  this movie. It was a good movie"]

In [58]:
predictor.predict(data)

['neg', 'neg', 'pos']